# Reproducing and trying to solve the bug in [this ticket](https://github.com/orgs/OpenMined/projects/81/views/1?pane=issue&itemId=28179093) - first notebook

In [1]:
import syft as sy
sy.requires(">=0.8,<0.8.1")

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/dk/Desktop/projects/PySyft


✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8 and the requirement <0.8.1


In [2]:
node = sy.orchestra.launch(name="test-domain-1", port=8080, dev_mode=True, reset=True)

Starting test-domain-1 server on 0.0.0.0:8080

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite



INFO:     Started server process [187294]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:52700 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:52700 - "POST /api/v1/new/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:52700 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:52716 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52720 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:52728 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:57328 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:55720 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:55728 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:55744 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:45012 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:45018 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
I

In [3]:
domain_client = node.login(email="info@openmined.org", password="changethis")

#### Construct a `custom policy`

In [4]:
from typing import List, Dict, Any, Optional

class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict

In [5]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])

In [6]:
print(f"{policy.n_calls = }")
print(f"{policy.downloadable_output_args = }")

policy.n_calls = 2
policy.downloadable_output_args = ['y']


In [7]:
policy.init_kwargs

{'n_calls': 1, 'downloadable_output_args': ['y']}

#### Make a pointer

In [8]:
import numpy as np
x = np.array([1,2,3])
x_pointer = sy.ActionObject.from_obj(x)
x_pointer

```python
Pointer
```
array([1, 2, 3])

In [9]:
domain_client.api.services.action.save(x_pointer)

<class 'syft.service.response.SyftSuccess'>: <class 'syft.service.action.numpy.NumpyArrayObject'> saved

In [10]:
x_pointer.id

<UID: b9eff9bac123452588e7f41f989f0faf>

#### Make a function with the `RepeatedCallPolicy`

In [11]:
@sy.syft_function(
    input_policy=sy.ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, 
                                     downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}

`@sy.syft_function` turns `func` into a `SubmitUserCode`

In [19]:
@sy.syft_function??

In [15]:
type(func), func.input_policy_type, func.output_policy_type, func.input_policy_init_kwargs, func.output_policy_init_kwargs

(syft.service.code.user_code.SubmitUserCode,
 syft.service.policy.policy.ExactMatch,
 syft.service.policy.policy.SubmitUserPolicy,
 {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'x': <UID: b9eff9bac123452588e7f41f989f0faf>}},
 {'n_calls': 10, 'downloadable_output_args': ['y']})

In [16]:
original_output_policy = func.output_policy_type

In [17]:
original_output_policy

```python
class SubmitUserPolicy:
  id: str = None
  init_kwargs: str = {'code': 'class RepeatedCallPolicy(sy.CustomOutputPolicy):\n    n_calls: int = 0\n    downloadable_output_args: List[str] = []\n    state: Dict[Any, Any] = {}\n\n    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):\n        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []\n        self.n_calls = n_calls + 1\n        self.state = {"counts": 0}\n\n    def public_state(self):\n        return self.state["counts"]\n        \n    def apply_output(self, context, outputs):\n        output_dict = {}\n        if self.state["counts"] < self.n_calls:\n            for output_arg in self.downloadable_output_args:\n                output_dict[output_arg] = outputs[output_arg]\n\n            self.state["counts"] += 1\n        else:\n            return None\n\n        return output_dict\n', 'class_name': 'RepeatedCallPolicy', 'input_kwargs': ('n_calls', 'downloadable_output_args')}
  code: str = "class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict
"
  class_name: str = "RepeatedCallPolicy"
  input_kwargs: str = ['n_calls', 'downloadable_output_args']

```

In [18]:
print(original_output_policy.code)

class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict



In [20]:
from syft.serde.recursive import TYPE_BANK

In [21]:
TYPE_BANK

{'numpy.ndarray': (True,
  <function syft.serde.arrow.numpy_serialize(obj: numpy.ndarray) -> bytes>,
  <function syft.serde.arrow.numpy_deserialize(buf: bytes) -> numpy.ndarray>,
  None,
  [],
  {},
  numpy.ndarray),
 'numpy._globals._NoValueType': (False,
  <function syft.serde.recursive.rs_object2proto(self: Any) -> capnp.lib.capnp._DynamicStructBuilder>,
  <function syft.serde.recursive.rs_proto2object(proto: capnp.lib.capnp._DynamicStructBuilder) -> Any>,
  None,
  [],
  {},
  numpy._globals._NoValueType),
 'numpy.bool_': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.bool_),
 'numpy.int8': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int8),
 'numpy.int16': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int16),
 'numpy.int32': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int32),
 'numpy.int64': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int64),
 'numpy.uint8': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint8),
 'numpy.uint16': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint16),
 'numpy.uint32': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint32),
 'numpy.uint64': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint64),
 'numpy.float32': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.float32),
 'numpy.float64': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.float64),
 'numpy.float16': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.float16),
 'builtins.int': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x_bytes)>,
  None,
  [],
  {},
  int),
 'builtins.float': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  None,
  [],
  {},
  float),
 'builtins.bytes': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  None,
  [],
  {},
  bytes),
 'builtins.str': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  None,
  [],
  {},
  str),
 'builtins.list': (True,
  <function syft.serde.recursive_primitives.serialize_iterable(iterable: Collection) -> bytes>,
  functools.partial(<function deserialize_iterable at 0x7f6ca5168790>, <class 'list'>),
  None,
  [],
  {},
  list),
 'builtins.tuple': (True,
  <function syft.serde.recursive_primitives.serialize_iterable(iterable: Collection) -> bytes>,
  functools.partial(<function deserialize_iterable at 0x7f6ca5168790>, <class 'tuple'>),
  None,
  [],
  {},
  tuple),
 'builtins.dict': (True,
  <function syft.serde.recursive_primitives.serialize_kv(map: Mapping) -> bytes>,
  functools.partial(<function deserialize_kv at 0x7f6ca5168940>, <class 'dict'>),
  None,
  [],
  {},
  dict),
 'collections.defaultdict': (True,
  <function syft.serde.recursive_primitives.serialize_defaultdict(df_dict: collections.defaultdict) -> bytes>,
  <function syft.serde.recursive_primitives.deserialize_defaultdict(blob: bytes) -> Mapping>,
  None,
  [],
  {},
  collections.defaultdict

there is no `RepeatedCallPolicy` in the above `TYPE_BANK`

#### Request code execution

In [22]:
domain_client.api.services.code.request_code_execution??

In [24]:
domain_client.api.services.code.request_code_execution(func)

```python
class Request:
  id: str = c3025f2fe9d24fe8b275d68506f5f550
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-05-18 20:32:51
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "b42eb00d702416b1cbdbd55aea147498165c458f23b9f79b660da145cdd31356"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

In [30]:
from syft import MessageStatus
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,c61b85be7e4f441e8065b076cf65c50d,Approval Request,MessageStatus.UNDELIVERED,2023-05-18 20:32:51,<<class 'syft.service.request.request.Request'...


In [31]:
from syft.service.request.request import ObjectMutation, UserCodeStatusChange, Request
func = None
request = None
for message in messages:
    req = message.linked_obj.resolve
    if isinstance(req, Request):
        for change in req.changes:
            if isinstance(change, UserCodeStatusChange):
                user_code = change.linked_obj.resolve
                if "func" in user_code.service_func_name:
                    func = user_code
                    request = req

In [32]:
request

```python
class Request:
  id: str = c3025f2fe9d24fe8b275d68506f5f550
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-05-18 20:32:51
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "b42eb00d702416b1cbdbd55aea147498165c458f23b9f79b660da145cdd31356"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

In [33]:
request.changes[0]

```python
class UserCodeStatusChange:
  id: str = f213809059e445c280a6f6410584d551
  linked_obj: str = <<class 'syft.service.code.user_code.UserCode'>: 68e29d20afab45c094dc6f290d89fa49@<Node: 7bca415d13ed1ec841f0d0aede098dbb>
  value: str = UserCodeStatus.EXECUTE
  match_type: str = True

```

In [34]:
linked_obj = request.changes[0].linked_obj
linked_obj

```python
class LinkedObject:
  id: str = 4492de0684e54c33a4ad8b6ffa176780
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  service_type: str = <class 'syft.service.code.user_code_service.UserCodeService'>
  object_type: str = <class 'syft.service.code.user_code.UserCode'>
  object_uid: str = 68e29d20afab45c094dc6f290d89fa49

```

In [35]:
func = linked_obj.resolve

In [36]:
original_output_policy

```python
class SubmitUserPolicy:
  id: str = None
  init_kwargs: str = {'code': 'class RepeatedCallPolicy(sy.CustomOutputPolicy):\n    n_calls: int = 0\n    downloadable_output_args: List[str] = []\n    state: Dict[Any, Any] = {}\n\n    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):\n        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []\n        self.n_calls = n_calls + 1\n        self.state = {"counts": 0}\n\n    def public_state(self):\n        return self.state["counts"]\n        \n    def apply_output(self, context, outputs):\n        output_dict = {}\n        if self.state["counts"] < self.n_calls:\n            for output_arg in self.downloadable_output_args:\n                output_dict[output_arg] = outputs[output_arg]\n\n            self.state["counts"] += 1\n        else:\n            return None\n\n        return output_dict\n', 'class_name': 'RepeatedCallPolicy', 'input_kwargs': ('n_calls', 'downloadable_output_args')}
  code: str = "class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict
"
  class_name: str = "RepeatedCallPolicy"
  input_kwargs: str = ['n_calls', 'downloadable_output_args']

```

In [37]:
print(original_output_policy.code)

class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict



In [40]:
func.output_policy_type

```python
class UserPolicy:
  id: str = f7c1f23d4f8b42b595d94f460e51bb0c
  init_kwargs: str = {'code': 'class RepeatedCallPolicy(sy.CustomOutputPolicy):\n    n_calls: int = 0\n    downloadable_output_args: List[str] = []\n    state: Dict[Any, Any] = {}\n\n    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):\n        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []\n        self.n_calls = n_calls + 1\n        self.state = {"counts": 0}\n\n    def public_state(self):\n        return self.state["counts"]\n        \n    def apply_output(self, context, outputs):\n        output_dict = {}\n        if self.state["counts"] < self.n_calls:\n            for output_arg in self.downloadable_output_args:\n                output_dict[output_arg] = outputs[output_arg]\n\n            self.state["counts"] += 1\n        else:\n            return None\n\n        return output_dict\n', 'class_name': 'RepeatedCallPolicy', 'input_kwargs': ('n_calls', 'downloadable_output_args')}
  node_uid: str = None
  user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  raw_code: str = "class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict
"
  parsed_code: str = "from __future__ import annotations as annotations
import syft as sy
from typing import Any as Any
from typing import Callable as Callable
from typing import ClassVar as ClassVar
from typing import Dict as Dict
from typing import List as List
from typing import Optional as Optional
from typing import Set as Set
from typing import Tuple as Tuple
from typing import Type as Type

@sy.serializable()
class RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69(sy.UserOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __user_init__(self, n_calls=1, downloadable_output_args: List[str]=None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {'counts': 0}

    def public_state(self):
        return self.state['counts']

    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state['counts'] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]
            self.state['counts'] += 1
        else:
            return None
        return output_dict
    __module__ = 'syft.user'
    __canonical_name__ = 'RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69'
    __version__ = 1"
  signature: str = (n_calls, downloadable_output_args)
  class_name: str = "RepeatedCallPolicy"
  unique_name: str = "RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69"
  code_hash: str = "ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69"
  status: str = UserPolicyStatus.SUBMITTED

```

In [45]:
print(func.output_policy_type.parsed_code)

from __future__ import annotations as annotations
import syft as sy
from typing import Any as Any
from typing import Callable as Callable
from typing import ClassVar as ClassVar
from typing import Dict as Dict
from typing import List as List
from typing import Optional as Optional
from typing import Set as Set
from typing import Tuple as Tuple
from typing import Type as Type

@sy.serializable()
class RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69(sy.UserOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __user_init__(self, n_calls=1, downloadable_output_args: List[str]=None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {'counts': 0}

    def public_state(self):
        return self.state['counts

In [46]:
func.output_policy_state

b''

In [47]:
print(f"{x_pointer = }")
result = func.unsafe_function(x=x_pointer)
result

x_pointer = Pointer:
[1 2 3]


{'y': array([2, 3, 4])}

In [48]:
TYPE_BANK

{'numpy.ndarray': (True,
  <function syft.serde.arrow.numpy_serialize(obj: numpy.ndarray) -> bytes>,
  <function syft.serde.arrow.numpy_deserialize(buf: bytes) -> numpy.ndarray>,
  None,
  [],
  {},
  numpy.ndarray),
 'numpy._globals._NoValueType': (False,
  <function syft.serde.recursive.rs_object2proto(self: Any) -> capnp.lib.capnp._DynamicStructBuilder>,
  <function syft.serde.recursive.rs_proto2object(proto: capnp.lib.capnp._DynamicStructBuilder) -> Any>,
  None,
  [],
  {},
  numpy._globals._NoValueType),
 'numpy.bool_': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.bool_),
 'numpy.int8': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int8),
 'numpy.int16': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int16),
 'numpy.int32': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int32),
 'numpy.int64': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int64),
 'numpy.uint8': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint8),
 'numpy.uint16': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint16),
 'numpy.uint32': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint32),
 'numpy.uint64': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint64),
 'numpy.float32': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.float32),
 'numpy.float64': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.float64),
 'numpy.float16': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.float16),
 'builtins.int': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x_bytes)>,
  None,
  [],
  {},
  int),
 'builtins.float': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  None,
  [],
  {},
  float),
 'builtins.bytes': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  None,
  [],
  {},
  bytes),
 'builtins.str': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  None,
  [],
  {},
  str),
 'builtins.list': (True,
  <function syft.serde.recursive_primitives.serialize_iterable(iterable: Collection) -> bytes>,
  functools.partial(<function deserialize_iterable at 0x7f6ca5168790>, <class 'list'>),
  None,
  [],
  {},
  list),
 'builtins.tuple': (True,
  <function syft.serde.recursive_primitives.serialize_iterable(iterable: Collection) -> bytes>,
  functools.partial(<function deserialize_iterable at 0x7f6ca5168790>, <class 'tuple'>),
  None,
  [],
  {},
  tuple),
 'builtins.dict': (True,
  <function syft.serde.recursive_primitives.serialize_kv(map: Mapping) -> bytes>,
  functools.partial(<function deserialize_kv at 0x7f6ca5168940>, <class 'dict'>),
  None,
  [],
  {},
  dict),
 'collections.defaultdict': (True,
  <function syft.serde.recursive_primitives.serialize_defaultdict(df_dict: collections.defaultdict) -> bytes>,
  <function syft.serde.recursive_primitives.deserialize_defaultdict(blob: bytes) -> Mapping>,
  None,
  [],
  {},
  collections.defaultdict

No `RepeatedCallPolicy` in the `TYPE_BANK` yet

#### Switch to another noteobook or ipython shell and run the code below
```python
import syft as sy

my_domain_client = sy.login(email="info@openmined.org", password="changethis", port=8080)

from syft.client.api import SyftAPICall

api_call = SyftAPICall(
    node_uid=my_domain_client.id, path="request.get_all", args=(), kwargs={}, blocking=True
)

signed_api_call = api_call.sign(my_domain_client.credentials)

response = my_domain_client.connection.make_call(signed_call=signed_api_call)

response.message
```

#### Until this point, there will be no bug. However, the bug will occur after we call `request.accept_by_depositing_result(result)`

In [49]:
final_result = request.accept_by_depositing_result(result) 
final_result

<class 'syft.service.response.SyftSuccess'>: Request c3025f2fe9d24fe8b275d68506f5f550 changes applied

In [50]:
user_code_func = change.linked_obj.resolve
type(user_code_func)

syft.service.code.user_code.UserCode

In [51]:
user_code_func.output_policy_state

b'\x00\x00\x00\x00\xce\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00]\x00\x00\x00F\x00\x00\x00y\x00\x00\x00F\x00\x00\x00\x05\x00\x00\x00\xfa\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00syft.user.RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69\x00\x00=\x00\x00\x00\xca\x00\x00\x00\xad\x00\x00\x00\x1a\x00\x00\x00\x11\x01\x00\x00b\x00\x00\x00M\x01\x00\x00B\x00\x00\x00y\x01\x00\x00J\x00\x00\x00\xad\x01\x00\x00z\x00\x00\x00\xe5\x01\x00\x00r\x00\x00\x00\x1d\x02\x00\x002\x00\x00\x00-\x00\x00\x00\x0e\x00\x00\x00\x91\x00\x00\x00\x0e\x00\x00\x00\xf9\x00\x00\x00\x0e\x00\x00\x001\x01\x00\x00\x0e\x00\x00\x00a\x01\x00\x00\x0e\x00\x00\x00\x95\x01\x00\x00\x0e\x00\x00\x00\xcd\x01\x00\x00\x0e\x00\x00\x00\x01\x02\x00\x00\x0e\x00\x00\x00downloadable_output_args\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x06\x00\x00\x00\x00\x00\x00\x17\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00

There is `syft.user.RepeatedCallPolicy_<hash>` in `user_code_func.output_policy_state`

#### Possibly the issue is in `output_policy` property (UserCode.output_policy in `syft/service/code/user_code.py`) which initializes the `output_policy` and adds the serialized class to `output_policy_state`

In [55]:
user_code_func.output_policy??

In [56]:
type(user_code_func.output_policy)

syft.user.RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69

In [57]:
domain_client.api.services.messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,bfccabffbb244fc09e56f7576d8cff1c,Approval Request,MessageStatus.UNDELIVERED,2023-05-18 20:39:36,<<class 'syft.service.request.request.Request'...
1,syft.service.message.messages.Message,c61b85be7e4f441e8065b076cf65c50d,Approval Request,MessageStatus.UNDELIVERED,2023-05-18 20:32:51,<<class 'syft.service.request.request.Request'...


In [58]:
TYPE_BANK

{'numpy.ndarray': (True,
  <function syft.serde.arrow.numpy_serialize(obj: numpy.ndarray) -> bytes>,
  <function syft.serde.arrow.numpy_deserialize(buf: bytes) -> numpy.ndarray>,
  None,
  [],
  {},
  numpy.ndarray),
 'numpy._globals._NoValueType': (False,
  <function syft.serde.recursive.rs_object2proto(self: Any) -> capnp.lib.capnp._DynamicStructBuilder>,
  <function syft.serde.recursive.rs_proto2object(proto: capnp.lib.capnp._DynamicStructBuilder) -> Any>,
  None,
  [],
  {},
  numpy._globals._NoValueType),
 'numpy.bool_': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.bool_),
 'numpy.int8': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int8),
 'numpy.int16': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int16),
 'numpy.int32': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int32),
 'numpy.int64': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.int64),
 'numpy.uint8': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint8),
 'numpy.uint16': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint16),
 'numpy.uint32': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint32),
 'numpy.uint64': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.uint64),
 'numpy.float32': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.float32),
 'numpy.float64': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.float64),
 'numpy.float16': (True,
  <function syft.serde.array.<lambda>(x)>,
  <function syft.serde.array.<lambda>(buffer)>,
  None,
  [],
  {},
  numpy.float16),
 'builtins.int': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x_bytes)>,
  None,
  [],
  {},
  int),
 'builtins.float': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  None,
  [],
  {},
  float),
 'builtins.bytes': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  None,
  [],
  {},
  bytes),
 'builtins.str': (True,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  <function syft.serde.recursive_primitives.<lambda>(x)>,
  None,
  [],
  {},
  str),
 'builtins.list': (True,
  <function syft.serde.recursive_primitives.serialize_iterable(iterable: Collection) -> bytes>,
  functools.partial(<function deserialize_iterable at 0x7f6ca5168790>, <class 'list'>),
  None,
  [],
  {},
  list),
 'builtins.tuple': (True,
  <function syft.serde.recursive_primitives.serialize_iterable(iterable: Collection) -> bytes>,
  functools.partial(<function deserialize_iterable at 0x7f6ca5168790>, <class 'tuple'>),
  None,
  [],
  {},
  tuple),
 'builtins.dict': (True,
  <function syft.serde.recursive_primitives.serialize_kv(map: Mapping) -> bytes>,
  functools.partial(<function deserialize_kv at 0x7f6ca5168940>, <class 'dict'>),
  None,
  [],
  {},
  dict),
 'collections.defaultdict': (True,
  <function syft.serde.recursive_primitives.serialize_defaultdict(df_dict: collections.defaultdict) -> bytes>,
  <function syft.serde.recursive_primitives.deserialize_defaultdict(blob: bytes) -> Mapping>,
  None,
  [],
  {},
  collections.defaultdict

We can see `syft.user.RepeatedCallPolicy_<hash>` added in the above `TYPE_BANK`

#### Let's try serializing a `CustomOutputPolicy`

In [23]:
import syft as sy
from syft.service.policy.policy import CustomPolicy
from syft.service.policy.policy import CustomOutputPolicy

We can `serialize` and `deserialize` the `CustomPolicy`

In [39]:
byte = sy.serialize(CustomPolicy, to_bytes=True)
back = sy.deserialize(byte, from_bytes=True)
back == CustomPolicy

True

In [37]:
byte = sy.serialize(CustomOutputPolicy, to_bytes=True)
back = sy.deserialize(byte, from_bytes=True)
back

syft.service.policy.policy.CustomOutputPolicy

In [26]:
CustomPolicy

syft.service.policy.policy.CustomPolicy

In [24]:
sy.deserialize(sy.serialize(policy, to_bytes=True), 
               from_bytes=True)

In [36]:
RepeatedCallPolicy.__class__.__module__

syft.service.policy.policy.CustomPolicy

In [24]:
import syft as sy

class a(type):
    print('a')
    
class b(metaclass=a):
    print('b')
    
# a == sy.deserialize(sy.serialize(a, to_bytes=True), 
#                from_bytes=True)
b == sy.deserialize(sy.serialize(b, to_bytes=True), 
               from_bytes=True)

a
b


Exception: __main__.a not in TYPE_BANK